# 0. Contents

1. importing libraries 

2. Creating price_range columns

3. Creating price_range columns with different method

4. Creating busiest day column

5. Creating busiest_days column

6. Creating busiest time of day column

# 1. Importing Libraries and Data

In [1]:
# importing libaraies
import pandas as pd
import numpy as np
import os

In [2]:
# creating main path
path = r'C:\Users\muffi\OneDrive\CareerFoundry\Instacart Basket Analysis 2023'

In [3]:
# importing ords and prods merged data
df_ords_prods_merged = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged.pkl'))

In [10]:
# dropping the merge flag col
df_ords_prods_merged = df_ords_prods_merged.drop(columns = ['_merge'])

In [12]:
# checking the importing df 
df_ords_prods_merged.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20,0
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,0.0,10,0
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11,1


# 2. If-Statements with User-Defined Functions

In [14]:
# creating a subset of data to work with
df_sub = df_ords_prods_merged[:1000000]

In [21]:
# creating a function that aggregates products according to price
# price <= $5 is low-range
# $5 < price <= $15 is mid-range
# $15 < price is high-range

def price_label(row):
    if row['prices'] <= 5:
        return 'Low-range product'
    elif (row['prices'] > 5) and (row['prices'] <= 15):
        return 'Mid-range product'
    elif row['prices'] > 15:
        return 'High-range product'
    else:
        return 'Not enough data'

In [22]:
# creatind price_range col using the price_label function
df_sub['price_range'] = df_sub.apply(price_label, axis=1)

C:\Users\muffi\AppData\Local\Temp\ipykernel_23220\3148117953.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['price_range'] = df_sub.apply(price_label, axis=1)


In [23]:
# aggregating the results of the price_range col
df_sub['price_range'].value_counts()

Mid-range product     652638
Low-range product     338018
High-range product      9344
Name: price_range, dtype: int64

In [24]:
# checking the max price 
df_sub['prices'].max()

24.5

# 3. If-Statements with .loc() Function

In [28]:
# .loc() col for low-range products 
df_sub.loc[df_sub['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [29]:
# .loc() col for mid-range products 
df_sub.loc[(df_sub['prices'] > 5) & (df_sub['prices'] <= 15), 'price_range_loc'] = 'Mid-range product'

In [30]:
# .loc() col for high-range products 
df_sub.loc[df_sub['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [31]:
df_sub['price_range_loc'].value_counts()

Mid-range product     652638
Low-range product     338018
High-range product      9344
Name: price_range_loc, dtype: int64

Got the same results without using the function and much faster 

In [33]:
# .loc() col for low-range products for the whole df
df_ords_prods_merged.loc[df_ords_prods_merged['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [35]:
# .loc() col for mid-range products for the whole df
df_ords_prods_merged.loc[(df_sub['prices'] > 5) & (df_ords_prods_merged['prices'] <= 15), 'price_range_loc'] = 'Mid-range product'

In [36]:
# .loc() col for high-range products for the whole df
df_ords_prods_merged.loc[df_ords_prods_merged['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [38]:
# checkign the price_range col for the whole df
df_ords_prods_merged['price_range_loc'].value_counts()

Low-range product     10126321
Mid-range product       652638
High-range product      417678
Name: price_range_loc, dtype: int64

# 4. If-Statements with For-Loops

In [40]:
# frequency table of the orders each day of the week
# the week starts at Saturday with 0
df_ords_prods_merged['orders_day_of_week'].value_counts(dropna = False)

0    6204182
1    5660230
6    4496490
2    4213830
5    4205791
3    3840534
4    3783802
Name: orders_day_of_week, dtype: int64

In [48]:
# creating a blank results table
result = []
# for-loop that goes through each cell in orers_day_of_week and records the results
# Saturday (0) is the busiest day
# Wedenesday (4) is the lest busy day
# everything other number is regularly busy 
for value in df_ords_prods_merged['orders_day_of_week']:
    if value == 0:
        result.append('Busiest day')
    elif value == 4:
        result.append('Least busy')
    else:
        result.append('Regularly busy')

In [49]:
# checking the results list
result[:10]

['Regularly busy',
 'Regularly busy',
 'Busiest day',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy']

In [46]:
# combing the results list and the merged df
df_ords_prods_merged['busiest day'] = result

In [47]:
# frequency of the busist_day col 
df_ords_prods_merged['busiest day'].value_counts()

Regularlly busy    22416875
Busiest day         6204182
Least busy          3783802
Name: busiest day, dtype: int64

The numbers match those of the previous frequency table

# 5. "Busiest days" Column

Column aggregating the 2 busiest day, the 2 slowest days, and the rest of the days

In [51]:
# creating a blank results table
busy_results = []

# Sat (0) and Sun (1) are the "Busiest days" 
# Wed (4) and Tue (3) are the "Least busy" 
# the rest are "Regularly busy"

# for loop recording the busy levels
for value in df_ords_prods_merged['orders_day_of_week']:
    if (value == 0) or (value == 1):
        busy_results.append('Busiest day')
    elif (value == 4) or (value == 3):
        busy_results.append('Least busy')
    else:
        busy_results.append('Regularly busy')

In [53]:
# merging df with reslts
df_ords_prods_merged['busiest_days'] = busy_results

In [59]:
# checking the results with the freq table
df_ords_prods_merged['busiest_days'].value_counts()

Regularly busy    12916111
Busiest day       11864412
Least busy         7624336
Name: busiest_days, dtype: int64

The values in the new columns match those found in the origina freqeuncy table.

# 6. Busiest Time of the Day

Column aggregating th period of time with the most orders, average orders, and least orders

I am assuming "average" means 1 standard deviation above or below the statistical average

In [75]:
# calculating avg order time 
avg_ords = df_ords_prods_merged['order_hour_of_day'].value_counts().mean()

In [84]:
# calculating the order std
stdv_ords = df_ords_prods_merged['order_hour_of_day'].value_counts().std()

In [111]:
# calculating the upper bounds for "average" orders
upper_avg = avg_ords + stdv_ords
# viewing the upper average
upper_avg

2428571.0387057555

In [112]:
# calculating the lower bounds for "average" orders
lower_avg = avg_ords - stdv_ords
# viewing the lower average
lower_avg

271833.877960911

In [94]:
df_ords_prods_merged['order_hour_of_day'].value_counts()

10    2761760
11    2736140
14    2689136
15    2662144
13    2660954
12    2618532
16    2535202
9     2454203
17    2087654
8     1718118
18    1636502
19    1258305
20     976156
7      891054
21     795637
22     634225
23     402316
6      290493
0      218769
1      115700
5       87961
2       69375
4       53242
3       51281
Name: order_hour_of_day, dtype: int64

In [ ]:
# 10, 11, 14, 15, 13, 12, 16 have the most orders 
# 0, 1, 5, 2, 4, and 3 have the fewest orders 
# the rest have average orders 

# creting the empty result list 
order_results = []

# creating the results list 
for val in df_ords_prods_merged['order_hour_of_day']:
    if (val >= 10) and (val <= 16):
        order_results.append('Most orders')
    elif (val >= 0) and (val <= 5):
        order_results.append('Fewst orders')
    else:
        order_results.append('Average orders')

In [126]:
# merging the results to the main df
df_ords_prods_merged['busiest_period_of_day'] = order_results

In [128]:
# freq of busiest_period_of_da
df_ords_prods_merged['busiest_period_of_day'].value_counts()

Most orders       18663868
Average orders    13144663
Fewst orders        596328
Name: busiest_period_of_day, dtype: int64

The values in the new fequency tble match those from the original one

# 7. Exporting Files

In [133]:
df_ords_prods_merged.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_v2.pkl'))